In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train=pd.read_csv('train.tsv',sep='\t')

In [ ]:
df_train=df_train[0:50000]

In [5]:
df_train=df_train.dropna()

In [6]:
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [7]:
import nltk
import nltk.corpus
import nltk.stem.porter
porter_stemmer=nltk.stem.porter.PorterStemmer()
stopword=set(nltk.corpus.stopwords.words('english'))

In [8]:
bigl=[]
for string in df_train['Phrase']:
    lis=string.split()
    l=[]
    for word in lis:
        if word.isalpha()==True:
            #if word not in stopword:
            if len(word)!=1:
                word=word.lower()
                word=porter_stemmer.stem(word)
                l.append(word)
    bigl.append(l)

In [9]:
df_train['Feature Words']=bigl

In [10]:
from gensim.models.word2vec import Word2Vec

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [11]:
num_features = 1000  # Word vector dimensionality
min_word_count = 2 # Minimum word count
num_workers = 15    # Number of parallel threads
context = 5        # Context window size
downsampling = 1e-3 #

In [12]:
model = Word2Vec(df_train['Feature Words'],\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

In [13]:
for words in df_train['Feature Words']:
    for word in words:
        if word in stopword:
            words.remove(word)

In [14]:
index2word_set = set(model.wv.index2word)

In [15]:
##Function to get average of all word vectors in a line

def av_vector_line(line):
    vector_feature=np.zeros(num_features,dtype='float32')
    num_words=0
    for word in line:
        if word in index2word_set:
            vector_feature=np.add(vector_feature,model[word])
            num_words+=1
    
    vector_feature=np.divide(vector_feature,num_words)
    return vector_feature

In [16]:
##Function to get the av feature matrix for training 

def av_feature_matrix(lines):
    av_feature=[]
    
    for line in lines:
        av_feature.append(av_vector_line(line))
    
    av_feature=np.array(av_feature)
    return av_feature

In [17]:
train_data_matrix=av_feature_matrix(df_train['Feature Words'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [18]:
x=train_data_matrix
y=np.array(df_train['Sentiment'])

In [19]:
z=np.isnan(x)

In [20]:
a=[]
for i in range(len(z)):
    if z[i].all()==True:
        a.append(i)

In [21]:
x=np.delete(x,a,axis=0)
y=np.delete(y,a)    

In [22]:
import sklearn.datasets
import sklearn.linear_model
x_train,x_test,y_train,y_test=sklearn.model_selection.train_test_split(x,y,test_size=0.2)

In [23]:
labels=[]

for i in range(1000):
    labels.append(str(i))

In [24]:
x_dict_train={}
for i in range(len(labels)):
    l=[]
    for j in range(len(x_train)):
        l.append(x_train[j][i])
    x_dict_train[labels[i]]=l

In [25]:
feature_columns=[]
for key in x_dict_train.keys():
    feature_columns.append(tf.contrib.layers.feature_column.real_valued_column(key))

In [26]:
x_dict_test={}
for i in range(len(labels)):
    l=[]
    for j in range(len(x_test)):
        l.append(x_test[j][i])
    x_dict_test[labels[i]]=l

In [27]:
estimator = tf.estimator.DNNClassifier(feature_columns=feature_columns,hidden_units=[10,10],n_classes=5)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\rishi\\AppData\\Local\\Temp\\tmp98usgqjw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F00E729630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [28]:
def input_train():
    return x_dict_train,y_train

In [29]:
def input_test():
    return x_dict_test,y_test

In [30]:
estimator.train(input_fn=input_train,steps=500)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\rishi\AppData\Local\Temp\tmp98usgqjw\model.ckpt.
INFO:tensorflow:loss = 63384.984, step = 1
INFO:tensorflow:global_step/sec: 2.06137
INFO:tensorflow:loss = 48595.773, step = 101 (48.580 sec)
INFO:tensorflow:global_step/sec: 2.54804
INFO:tensorflow:loss = 47744.777, step = 201 (39.165 sec)
INFO:tensorflow:global_step/sec: 2.68906
INFO:tensorflow:loss = 47534.863, step = 301 (37.187 sec)
INFO:tensorflow:global_step/sec: 2.62226
INFO:tensorflow:loss = 47441.53, step = 401 (38.136 sec)
INFO:tensorflow:Saving checkpoints for 500 into C:\Users\rishi\AppData\Local\Temp\tmp98usgqjw\model.ckpt.
INFO:tensorflow:Loss for final step: 47324.77.


In [31]:
estimator.evaluate(input_fn=input_test,steps=200)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-22-17:50:22
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\rishi\AppData\Local\Temp\tmp98usgqjw\model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [20/200]
INFO:tensorflow:Evaluation [40/200]
INFO:tensorflow:Evaluation [60/200]
INFO:tensorflow:Evaluation [80/200]
INFO:tensorflow:Evaluation [100/200]
INFO:tensorflow:Evaluation [120/200]
INFO:tensorflow:Evaluation [140/200]
INFO:tensorflow:Evaluation [160/200]
INFO:tensorflow:Evaluation [180/200]
INFO:tensorflow:Evaluation [200/200]
INFO:tensorflow:Finished evaluation at 2018-07-22-17:50:42
INFO:tensorflow:Saving dict for global step 500: accuracy = 0.5377928, average_loss = 1.1961606, global_step = 500, loss = 11694.861
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 500: C:\Users\rishi\Ap

{'accuracy': 0.5377928,
 'average_loss': 1.1961606,
 'loss': 11694.861,
 'global_step': 500}